<a href="https://colab.research.google.com/github/theidari/alphabet_soup/blob/main/src/AlphabetSoupCharity_Optimization_tuner%20-%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="#880808"><h1><b>Alphabet Soup Charity Optimization</b></h1></font>
<p align="justify"><font color="#0A0888">
after examine and investigate diffrent parameter such as in <a href="https://github.com/theidari/alphabet_soup/blob/main/src/AlphabetSoupCharity_Optimization.ipynb"><font color="#FF5733"><buttom>parameter selection</font></a>
</font></p>
  <button class="btn success" id="myBtn3" onclick="window.location.href='https://theidari.github.io/'" style="width: 180px; border: 1px solid #cd5a4d; border-radius:5px; background-color: white; color: #cd5a4d; padding: 14px 28px; font-size: 16px; cursor: pointer; display: inline-block; margin-bottom:10px">Portfolio</button>

  <script>
      var buttons = document.querySelectorAll('.btn.success');

        buttons.forEach(function(button) {
          button.addEventListener('mouseover', function() {
            this.style.backgroundColor = '#cd5a4d';
            this.style.color = 'white';
            this.style.border = '1px solid white';
          });

          button.addEventListener('mouseout', function() {
            this.style.backgroundColor = 'white';
            this.style.color = '#cd5a4d';
            this.style.border = '1px solid #cd5a4d';
          });
        });
   </script>

In [1]:
# Delete the existing directory
!rm -rf alphabet_soup

# Clone the repository to a new directory
!git clone https://github.com/theidari/alphabet_soup.git

# Dependencies and setup
from alphabet_soup.src.package.constants import * # constants
from alphabet_soup.src.package.helpers import * # liberaries and functions

Cloning into 'alphabet_soup'...
remote: Enumerating objects: 285, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 285 (delta 39), reused 1 (delta 0), pack-reused 227
Receiving objects: 100% (285/285), 281.14 KiB | 2.25 MiB/s, done.
Resolving deltas: 100% (192/192), done.
☑ constants is imporetd
☑ helpers is imporetd


In [2]:
# Loading the data into a Pandas DataFrame
application_df = pd.read_csv(DATA_URL)

In [3]:
# Drop the 'EIN', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'STATUS' columns and keep "NAME".
application_df = application_df.drop(["EIN", "SPECIAL_CONSIDERATIONS", "ASK_AMT", "STATUS"], axis=1)

In [4]:
binning (application_df,"CLASSIFICATION",100)


--------------------------------------------------------------------------------
 Value Count before binning:
--------------------------------------------------------------------------------
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
--------------------------------------------------------------------------------
Value Count after binning:
--------------------------------------------------------------------------------
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64


In [5]:
binning (application_df,"APPLICATION_TYPE",100)


--------------------------------------------------------------------------------
 Value Count before binning:
--------------------------------------------------------------------------------
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
--------------------------------------------------------------------------------
Value Count after binning:
--------------------------------------------------------------------------------
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64


In [6]:
binning (application_df,"NAME",100)


--------------------------------------------------------------------------------
 Value Count before binning:
--------------------------------------------------------------------------------
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR              

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)

In [8]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(["IS_SUCCESSFUL"], axis=1)
y = application_numeric["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

input_features=len(X_train_scaled[1])

In [9]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice("activation",["relu","tanh","sigmoid"])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int("first_units",
        min_value=1,
        max_value=320,
        step=5), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int("num_layers", 1, 8)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int("units_" + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [10]:
!pip install -q -U keras-tuner
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=35,
    hyperband_iterations=2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 6.2 MB/s eta 0:00:00


In [11]:
tuner.search(X_train_scaled,y_train,epochs=35,validation_data=(X_test_scaled,y_test)) 

Trial 180 Complete [00h 02m 24s]
val_accuracy: 0.7528862953186035

Best val_accuracy So Far: 0.7552186846733093
Total elapsed time: 01h 22m 09s


In [12]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 251, 'num_layers': 8, 'units_0': 61, 'units_1': 16, 'units_2': 66, 'units_3': 91, 'units_4': 6, 'units_5': 81, 'units_6': 111, 'units_7': 21, 'tuner/epochs': 35, 'tuner/initial_epoch': 12, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0070'}
{'activation': 'relu', 'first_units': 306, 'num_layers': 1, 'units_0': 21, 'units_1': 41, 'units_2': 1, 'units_3': 86, 'units_4': 81, 'units_5': 16, 'units_6': 16, 'units_7': 41, 'tuner/epochs': 12, 'tuner/initial_epoch': 4, 'tuner/bracket': 3, 'tuner/round': 2, 'tuner/trial_id': '0038'}
{'activation': 'relu', 'first_units': 286, 'num_layers': 5, 'units_0': 101, 'units_1': 61, 'units_2': 1, 'units_3': 96, 'units_4': 21, 'units_5': 41, 'units_6': 66, 'units_7': 36, 'tuner/epochs': 12, 'tuner/initial_epoch': 4, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0153'}


In [13]:
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4941 - accuracy: 0.7552 - 811ms/epoch - 3ms/step
Loss: 0.4940686523914337, Accuracy: 0.7552186846733093
268/268 - 1s - loss: 0.4977 - accuracy: 0.7550 - 584ms/epoch - 2ms/step
Loss: 0.4976674020290375, Accuracy: 0.7549854516983032
268/268 - 2s - loss: 0.4977 - accuracy: 0.7549 - 2s/epoch - 7ms/step
Loss: 0.4976622760295868, Accuracy: 0.7548688054084778


In [2]:
models = tuner.get_best_models(num_models=3)
best_model = models[1]

best_model.build()
best_model.summary()

NameError: ignored